# 6 задача
## Численно и аналитически решить смешанную задачу для квазилинейного уравнения теплопроводности с точностью 1e-4 и сравнить их значения в 11 точках при t  = 1

### Постановка задачи (вариант №1 задание №2)

#### Дифференциальная задача:

\begin{gather*}
   {\partial u \over \partial t} = {\partial \over \partial x}(u^2 {\partial u \over \partial x})\\
\end{gather*}

\begin{equation}
0 \leq x \leq 1 
\end{equation}

#### Краевые условия задачи:

\begin{equation}
\left.
  \begin{array}{ccc}
      u(x,0) = (1+x) / \sqrt{5}, \;\;\; 0 \leq x \leq 1\\
      u(0,t) = 1 / \sqrt{5-4t}, \;\;\; 0 < t \leq 1\\
      u(1,t) = 2 / \sqrt{5-4t}, \;\;\; 0 < t \leq 1\\
  \end{array}
\right\}
\end{equation}

#### Аналитическое решение:

\begin{equation}
u(x,t) = {1+x \over \sqrt{5-4t}}
\end{equation}

#### Разностная схема:

Чтобы учесть нелинейность коэффициента теплопроводности, будет использована следующая схема:

\begin{equation}
{u_l^{k+1}-u_l^n \over \tau} = {[(u_{l+1}^k)^2+(u_l^k)^2](u_{l+1}^{k+1}-u_l^{k+1}) \over 2h^2} - {[(u_l^k)^2+(u_{l-1}^k)^2](u_l^{k+1}-u_{l-1}^{k+1}) \over 2h^2}
\end{equation}

Перепишу систему в более удобном виде:
\begin{equation}
-{\tau \over 2h^2}[(u_l^k)^2+(u_{l-1}^k)^2]u_{l-1}^{k+1}
+(1+{\tau \over 2h^2}[u_{l-1}^k)^2-(u_{l+1}^k)^2])u_l^{k+1}
-{\tau \over 2h^2}[(u_l^k)^2+(u_{l+1}^k)^2]u_{l+1}^{k+1} 
=u_l^n
\end{equation}

Обозначив коэффициенты в этом уравнении как a,b,c,d, получаю трехдиагональную СЛАУ для итераций:

\begin{equation} 
a_l^k u_{l-1}^{k+1} + b_l^k u_l^{k+1} + c_l^k u_{l+1}^{k+1} = d_l^n, \; \; \; l \in [0, L]
\end{equation}

In [12]:
import numpy as np
import matplotlib.pyplot
import pandas as pd
import math as m
import scipy as sp
from scipy.sparse import dia_matrix

In [26]:
# сделать итерацию по нелинейному коэффициенту
def nextlay(prevlay, n_lay, time):
    upper_arr = [-tau/2/h**2*(prevlay[i]**2+prevlay[i+1]**2) for i in range(2,N-2)]
    middle_arr= [1+tau/2/h**2*(prevlay[i-1]**2+prevlay[i+1]**2) for i in range(2,N-2)]
    lower_arr = [-tau/2/h**2*(prevlay[i-1]**2+prevlay[i]**2) for i in range(2,N-2)]

    A = dia_matrix((np.array([lower_arr, middle_arr, upper_arr]), [-1, 0, 1]), shape=(N-1, N-1)).toarray()
    B = np.array([n_lay for l in range(0, N-1)])
    solution = np.linalg.solve(A, B) # слой после итерации
    return solution

In [27]:
# максимум относительной ошибки между итерациями
def lays_diff(prevlay, currlay):
    diff = (currlay-prevlay)/currlay
    diff = abs(diff)
    maxdiff = max(diff)
    return maxdiff

In [28]:
N = 5121
x0, x1 = 0, 1
Nt = 41
Tend = 1
tau = Tend / (Nt-1)
h = (x1-x0) / (N-1)

eps = 1e-4

start_conditions = np.array([(1+i*(x1-x0)/(N-1))/m.sqrt(5) for i in range(0,N)])

In [34]:
Nlay = start_conditions

# обход по временным слоям N
for i in range(0,Nt):
    klay = Nlay
    
    # обход по итерациям k, пока не достигнется нужная точность
    while lays_diff(klay,nextlay(klay,Nlay,i*tau)) < eps:
        klay = nextlay(klay,Nlay,i*tau)
    
    # после достижения нужной точности обновляем временной слой
    Nlay = nextlay(klay,Nlay,i*tau)

new_solution = Nlay

In [10]:
def create0b(step):
    return [current_solution[1] - boundary_condition0[step] * C[0]]

def create1b(step):
    return [current_solution[-2] - boundary_condition1[step] * B[-1]]

In [36]:
# точное решение t=1
exactsol = np.array([1+i/10 for i in range(0,11)])

In [40]:
sol64 = new_solution[::64]

In [20]:
sol128 = new_solution[::128]

In [28]:
sol256 = new_solution[::256]

In [40]:
sol512 = new_solution[::512]

In [65]:
pd.DataFrame([sol64,sol128,sol256,sol512],columns=(np.linspace(0,1,num=11)),index=[641,1281,2561,5121])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
641,1.000998,1.100132,1.200073,1.300303,1.400340,1.500863,1.600442,1.700778,1.800369,1.900528,2.000251
1281,1.000229,1.100181,1.200246,1.300160,1.400072,1.500110,1.600156,1.700311,1.800143,1.900232,2.000087
2561,1.000027,1.100061,1.200048,1.300067,1.400004,1.500010,1.600095,1.700084,1.800021,1.900098,2.000009
5121,1.000019,1.100002,1.200012,1.300005,1.400012,1.500009,1.600020,1.700004,1.800020,1.900024,2.000018


In [66]:
pd.DataFrame([(sol64-exactsol)/exactsol,(sol128-exactsol)/exactsol,(sol256-exactsol)/exactsol,(sol512-exactsol)/exactsol],columns=(np.linspace(0,1,num=11)),index=[641,1281,2561,5121])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
641,0.000998,0.000120,0.000061,0.000233,0.000243,0.000576,0.000276,0.000458,0.000205,0.000278,0.000125
1281,0.000229,0.000164,0.000205,0.000123,0.000052,0.000074,0.000097,0.000183,0.000080,0.000122,0.000043
2561,0.000027,0.000056,0.000040,0.000052,0.000003,0.000007,0.000060,0.000050,0.000012,0.000052,0.000004
5121,0.000019,0.000002,0.000010,0.000004,0.000009,0.000006,0.000013,0.000002,0.000011,0.000013,0.000009
